In [1]:
from datetime import datetime
import pandas as pd

In [ ]:
def load_data_local(file_path):
    with open(file_path, encoding='utf-8') as f:
        lines = f.readlines()
    parsed = [line.strip().split(';') for line in lines]
    max_len = max(len(row) for row in parsed)
    colnames = [f'col{i}' for i in range(max_len)]
    df = pd.DataFrame(parsed, columns=colnames)
    return df

In [8]:
file_path = "dilans_data.csv"
raw_df = load_data_local(file_path)
raw_df.head()

,col0,col1,col2,col3,col4,col5
0,2018-01-01 00:01:01,read,country_7,2458151261,SEO,North America
1,2018-01-01 00:03:20,read,country_7,2458151262,SEO,South America
2,2018-01-01 00:04:01,read,country_7,2458151263,AdWords,Africa
3,2018-01-01 00:04:02,read,country_7,2458151264,AdWords,Europe
4,2018-01-01 00:05:03,read,country_8,2458151265,Reddit,North America


In [ ]:
KNOWN_SOURCES = {'Reddit', 'SEO', 'AdWords'}

def parse_read_event(fields, i):
    dt = fields[i]
    country = fields[i+2]
    user_id = fields[i+3]

    if fields[i+4] in KNOWN_SOURCES:
        source = fields[i+4]
        topic = fields[i+5]
        return {
            'datetime': dt,
            'event_type': 'read',
            'country': country,
            'user_id': user_id,
            'source': source,
            'topic': topic
        }, 6
    else:
        topic = fields[i+4]
        return {
            'datetime': dt,
            'event_type': 'read',
            'country': country,
            'user_id': user_id,
            'source': 'Returning',
            'topic': topic
        }, 5

def parse_subscribe_event(fields, i):
    return {
        'datetime': fields[i],
        'event_type': 'subscribe',
        'user_id': fields[i+2]
    }, 3

def parse_buy_event(fields, i):
    return {
        'datetime': fields[i],
        'event_type': 'buy',
        'user_id': fields[i+2],
        'price': fields[i+3]
    }, 4

def parse_log_file(file_path):
    read_rows, subscribe_rows, buy_rows = [], [], []

    with open(file_path, encoding='utf-8') as f:
        for line in f:
            fields = line.strip().split(';')
            i = 0
            while i < len(fields):
                try:
                    event_type = fields[i+1]

                    if event_type == 'read':
                        record, step = parse_read_event(fields, i)
                        read_rows.append(record)
                    elif event_type == 'subscribe':
                        record, step = parse_subscribe_event(fields, i)
                        subscribe_rows.append(record)
                    elif event_type == 'buy':
                        record, step = parse_buy_event(fields, i)
                        buy_rows.append(record)
                    else:
                        step = 1

                    i += step

                except IndexError:
                    break

    return (
        clean_read_df(read_rows),
        clean_subscribe_df(subscribe_rows),
        clean_buy_df(buy_rows)
    )

def clean_read_df(rows):
    df = pd.DataFrame(rows)
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    return df

def clean_subscribe_df(rows):
    df = pd.DataFrame(rows)
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    return df

def clean_buy_df(rows):
    df = pd.DataFrame(rows)
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
    return df


In [32]:
read_df, subscribe_df, buy_df = parse_log_file("dilans_data.csv")

In [46]:
def validate_dataframes(read_df, subscribe_df, buy_df):
    print("=== BASIC STRUCTURE & NULLS ===")
    for df, name in zip([read_df, subscribe_df, buy_df], ['read', 'subscribe', 'buy']):
        print(f"\n{name.upper()} DF SHAPE: {df.shape}")
        print(f"{name} nulls:\n", df.isnull().sum())
        print(f"{name} dtypes:\n", df.dtypes)

    print("\n=== UNIQUE VALUES IN READ_DF ===")
    print("Countries:", read_df['country'].unique())
    print("Sources:", read_df['source'].unique())
    print("Topics:", read_df['topic'].unique())

    print("\n=== VALUE CHECKS ===")
    print("Invalid countries:\n", read_df[~read_df['country'].str.startswith("country_")])
    print("Invalid sources:\n", read_df[~read_df['source'].isin(['Reddit', 'AdWords', 'SEO', 'Returning'])])

    print("\n=== DUPLICATE CHECKS ===")
    print("Duplicate read events:", read_df.duplicated().sum())
    print("Duplicate subscriptions:", subscribe_df.duplicated().sum())

    print("\n=== USER ACTIVITY ===")
    print("Reads per user (top 10):\n", read_df['user_id'].value_counts().head(10))
    subscribers_not_readers = set(subscribe_df['user_id']) - set(read_df['user_id'])
    print("Subscribers not in read_df:", len(subscribers_not_readers))

    buyers = set(buy_df['user_id'])
    readers = set(read_df['user_id'])
    subscribers = set(subscribe_df['user_id'])
    print("Buyers not in read_df:", len(buyers - readers))
    print("Buyers not in subscribe_df:", len(buyers - subscribers))

    print("\n=== PRICE & TIMEFRAME ===")
    print("Price distribution:\n", buy_df['price'].value_counts())
    print("Read timeframe:", read_df['datetime'].min(), "->", read_df['datetime'].max())
    print("Subscribe timeframe:", subscribe_df['datetime'].min(), "->", subscribe_df['datetime'].max())
    print("Buy timeframe:", buy_df['datetime'].min(), "->", buy_df['datetime'].max())

    print("\n=== UNIQUENESS IN READ_DF ===")
    print("Total reads:", len(read_df))
    print("Unique user-topic pairs:", read_df[['user_id', 'topic']].drop_duplicates().shape[0])
    print("Unique user-country pairs:", read_df[['user_id', 'country']].drop_duplicates().shape[0])


In [47]:
validate_dataframes(read_df, subscribe_df, buy_df)

=== BASIC STRUCTURE & NULLS ===

READ DF SHAPE: (581877, 6)
read nulls:
 datetime      0
event_type    0
country       0
user_id       0
source        0
topic         0
dtype: int64
read dtypes:
 datetime      datetime64[ns]
event_type            object
country               object
user_id               object
source                object
topic                 object
dtype: object

SUBSCRIBE DF SHAPE: (7618, 3)
subscribe nulls:
 datetime      0
event_type    0
user_id       0
dtype: int64
subscribe dtypes:
 datetime      datetime64[ns]
event_type            object
user_id               object
dtype: object

BUY DF SHAPE: (8407, 4)
buy nulls:
 datetime      0
event_type    0
user_id       0
price         0
dtype: int64
buy dtypes:
 datetime      datetime64[ns]
event_type            object
user_id               object
price                  int64
dtype: object

=== UNIQUE VALUES IN READ_DF ===
Countries: ['country_7' 'country_8' 'country_6' 'country_2' 'country_5' 'country_4'
 'country_3